In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import collections
from matplotlib.pyplot import show

# our code (mark it at autoreload at every cell execution - useful in developement mode)
%load_ext autoreload
%autoreload 1
%aimport runners_utils

In [ ]:
PATH_TO_DATA = '../../Scraping/DataSport/Data/Runners/'
# change the name to the global like website but need to see if the website still works.
RUNNERS_FILE = 'runners_2009.csv'
RUNS_FILE = 'runs_2009.csv'

In [ ]:
runners = pd.read_csv(PATH_TO_DATA + RUNNERS_FILE)
runs = pd.read_csv(PATH_TO_DATA + RUNS_FILE)

In [ ]:
runners.head()

In [ ]:
runs.head()

We remove useless columns, and all value showed that the runners have resigned.

In [ ]:
runners_utils.preprocess_runners(runners)

In [ ]:
runs = runners_utils.remove_outliers(runs)

For each races we compute the distance, we remove all other sports than running to remain coherent during the study.

In [ ]:
runs['distance (km)'] = runs.apply(runners_utils.compute_distance_from_category,axis=1)
runs = runs[~runs['distance (km)'].isnull()]

For each runners, we compute the gender (male/female).

In [ ]:
result = runs.apply(runners_utils.compute_sex, args=(runners,), axis=1)

In order to have enought races for each runners, we take only runners who have run more than 100 races throught 1999 to 2016.

In [ ]:
runs = runners_utils.select_runners_by_numbers_of_runs(runs, 100)

For each runners, we compute his age at the moment of the race.

In [ ]:
runs['age'] = runs.apply(runners_utils.compute_age, args=(runners,), axis=1)

In [ ]:
runs['time (s)'] = runs.apply(runners_utils.transform_string_to_second, axis=1)
runs['speed (m/s)'] = runs['distance (km)']*1000 / runs['time (s)']

In [ ]:
runs['eventDate'] = runs.apply(runners_utils.compute_date_event, axis=1)

In [ ]:
runners_utils.remove_useless_columns(runs)

After all operations of preprocessing, we obtain 10 runners who have finished more than 100 running races.

In [ ]:
len(runs['acode'].unique())

In [ ]:
runs.head()

## Runner study.

In [ ]:
races_59 = runs[runs['acode'] == 'FU3KDU5G']
races_38 = runs[runs['acode'] == 'G756ZTL6']
races_26 = runs[runs['acode'] == 'FFJBKFSN']

In [ ]:
fig = plt.figure()
fig.tight_layout
fig.set_size_inches(15, 10)
fig.suptitle('Runnner speed by distance', fontsize=14)
annotation_plot2 = ['performance \n gap',(9.9,3), (9,4)]
runners_utils.presentation_performance_runners(fig, [races_59, races_38, races_26], annotation_plot2)

As we can notice from the graph above we see some enormous disparity between speed of race who have the same distance.
Futhermore an other interesting fact is the speed seems lower for the race of 31Km (http://www.sierre-zinal.com) than marathons.
The current study try to focus on vizualisation of multiple factors that could explain these gap:
 - Age
 - Training (As we don't have any personnal information, we take as reference how much they race)
 - Weather / altitude

### Training
--> faire un comparatif avec le nombre d'évenement au départ (Pour préciser que c'est des ultra sportifs) bc ont fait des triathlons ou du velo qui ne sont réferencer par la suite. (Pourrait réduire l'impact de la l"etude en précisant que c'est pas mondieur tout le monde quand meme)  

## TEST

In [ ]:
races_59 = runs[runs['acode'] ==  'FFJBKFSN'] #FU3KDU5G
races_38 = runs[runs['acode'] == 'G756ZTL6']
runs_42 = races_59[races_59['distance (km)'] == 42] 
group_by_age = races_59.groupby(['age'], sort=False).sum()
group_by_age.reset_index(inplace=True)

sns.boxplot(x="age", y="speed (m/s)", data=runs_42)


In [ ]:
group_by_age[['age','distance (km)']].plot(x='age', y='distance (km)')